# Deep Universal Regular Conditional Expectations:

---
This implements the universal deep neural model of $\mathcal{NN}_{1_{\mathbb{R}^n},\mathcal{D}}^{\sigma:\star}$ [Anastasis Kratsios](https://people.math.ethz.ch/~kratsioa/) - 2021.

---

## What does this code do?
1. Learn Heteroskedastic Non-Linear Regression Problem
     - $Y\sim f_{\text{unkown}}(x) + \epsilon$ where $f$ is an known function and $\epsilon\sim Laplace(0,\|x\|)$
2. Learn Random Bayesian Network's Law:
    - $Y = W_J Y^{J-1}, \qquad Y^{j}\triangleq \sigma\bullet A^{j}Y^{j-1} + b^{j}, \qquad Y^0\triangleq x$

3. In the above example if $A_j = M_j\odot \tilde{A_j}$ where $\tilde{A}_j$ is a deterministic matrix and $M_j$ is a "mask", that is, a random matrix with binary entries and $\odot$ is the Hadamard product then we recover the dropout framework.
4. Learn the probability distribution that the unique strong solution to the rough SDE with uniformly Lipschitz drivers driven by a factional Brownian motion with Hurst exponent $H \in [\frac1{2},1)$:
$$
X_t^x = x + \int_0^t \alpha(s,X_s^x)ds + \int_0^t \beta(s,X_s^x)dB_s^H
$$
belongs, at time $t=1$, to a ball about the initial point $x$ of random radius given by an independant exponential random-variable with shape parameter $\lambda=2$
5. Train a DNN to predict the returns of bitcoin with GD.  Since this has random initialization then each prediction of a given $x$ is stochastic...We learn the distribution of this conditional RV (conditioned on x in the input space).
$$
Y_x \triangleq \hat{f}_{\theta_{T}}(x), \qquad \theta_{(t+1)}\triangleq \theta_{(t)} + \lambda \sum_{x \in \mathbb{X}} \nabla_{\theta}\|\hat{f}_{\theta_t}(x) - f(x)\|, \qquad \theta_0 \sim N_d(0,1);
$$
$T\in \mathbb{N}$ is a fixed number of "SGD" iterations (typically identified by cross-validation on a single SGD trajectory for a single initialization) and where $\theta \in \mathbb{R}^{(d_{J}+1)+\sum_{j=0}^{J-1} (d_{j+1}d_j + 1)}$ and $d_j$ is the dimension of the "bias" vector $b_j$ defining each layer of the DNN with layer dimensions:
$$
\hat{f}_{\theta}(x)\triangleq A^{(J)}x^{(J)} + b^{(J)},\qquad x^{(j+1)}\triangleq \sigma\bullet A^{j}x^{(j)} + b^{j},\qquad x^{(0)}\triangleq x
.
$$

6. Extreme Learning Machines: 
    Just like the Bayesian network but then last layer is trained on the training set using KRidge!

#### Mode:
Software/Hardware Testing or Real-Deal?

In [1]:
trial_run = True

### Simulation Method:

In [2]:
# Random DNN
# f_unknown_mode = "Heteroskedastic_NonLinear_Regression"

# Random DNN internal noise
## Real-world data version
# f_unknown_mode = "Extreme_Learning_Machine"
### General Parameters
# activation_function == 'thresholding'
activation_function = 'sigmoid'
### Dataset Option 1
dataset_option = 'SnP'
### Dataset Option 2
# dataset_option = 'crypto'
Depth_Bayesian_DNN = 1
N_Random_Features = 10**2
## Simulated Data version
# f_unknown_mode = "DNN_with_Random_Weights"
width = 10**2

# Random Dropout applied to trained DNN
# f_unknown_mode = "DNN_with_Bayesian_Dropout"
Dropout_rate = 0.75

# GD with Randomized Input
# f_unknown_mode = "GD_with_randomized_input"
# GD_epochs = 50

# SDE with fractional Driver
f_unknown_mode = "Rough_SDE"
N_Euler_Steps = 50
Hurst_Exponent = 0.9

f_unknown_mode = "Rough_SDE_Vanilla"
## Define Process' dynamics in (2) cell(s) below.

---
#### More Meta-Parameters for "Vanilla" fractional SDE

In [3]:
## Monte-Carlo
N_Euler_Maruyama_Steps = 10**2

# End times for Time-Grid
T_end = 1
T_end_test = 1.1


## Grid
N_Grid_Finess = 1
Max_Grid = 0.5
x_0 = 1

# Number of Centers (\hat{\mu}_s)
N_Quantizers_to_parameterize = 1
N_Clusters = 2
Hurst_Exponent = 0.5
uniform_noise_level = 0

# Hyper-parameters of Cover
delta = 0.1
N_measures_per_center = 10**2

#### Vanilla Drift

In [4]:
def alpha(t,x):
    return .1*np.ones(problem_dim)#(.1-.5*(.01**2))*t + np.cos(x)

#### Vanilla Vol

In [5]:
def beta(t,x):
    return 0.01*np.ones(problem_dim)

---

## Problem Dimension

In [6]:
problem_dim = 1
if f_unknown_mode != 'Extreme_Learning_Machine':
    width = int(2*(problem_dim+1))

#### Vanilla fractional SDE:
If f_unknown_mode == "Rough_SDE_Vanilla" is selected, then we can specify the process's dynamics.  

In [7]:
# Depricated
# #--------------------------#
# # Define Process' Dynamics #
# #--------------------------#
# drift_constant = 0.1
# volatility_constant = 0.01

# # Define DNN Applier
# def f_unknown_drift_vanilla(x):
# #     x_internal = drift_constant*np.ones(problem_dim)
#     x_internal = drift_constant*np.sin(x)
#     return x_internal
# def f_unknown_vol_vanilla(x):
# #     x_internal = volatility_constant*np.diag(np.ones(problem_dim))
#     x_internal = volatility_constant*np.diag(np.cos(x))
#     return x_internal

## Note: *Why the procedure is so computationally efficient*?
---
 - The sample barycenters do not require us to solve for any new Wasserstein-1 Barycenters; which is much more computationally costly,
 - Our training procedure never back-propages through $\mathcal{W}_1$ since steps 2 and 3 are full-decoupled.  Therefore, training our deep classifier is (comparatively) cheap since it takes values in the standard $N$-simplex.

---

#### Grid Hyperparameter(s)
- Ratio $\frac{\text{Testing Datasize}}{\text{Training Datasize}}$.
- Number of Training Points to Generate

In [8]:
train_test_ratio = .1
N_train_size = 10**3

Monte-Carlo Paramters

In [9]:
## Monte-Carlo
N_Monte_Carlo_Samples = 10**3
N_Monte_Carlo_Samples_Test = 10**2 # How many MC-samples to draw from test-set?

Initial radis of $\delta$-bounded random partition of $\mathcal{X}$!

In [10]:
# Hyper-parameters of Cover
delta = 0.1
Proportion_per_cluster = .75

## Dependencies and Auxiliary Script(s)

In [11]:
# %run Loader.ipynb
exec(open('Loader.py').read())
# Load Packages/Modules
exec(open('Init_Dump.py').read())
import time as time #<- Note sure why...but its always seems to need 'its own special loading...'

Using TensorFlow backend.


Deep Feature Builder - Ready
Deep Classifier - Ready
Deep Feature Builder - Ready


# Simulate or Parse Data

In [12]:
#-------------------------------------------------------#
print("Generating/Prasing Data and Training MC-Oracle")
#-------------------------------------------------------#
if (f_unknown_mode != 'Rough_SDE') and (f_unknown_mode != 'Rough_SDE_Vanilla'):
    # %run Data_Simulator_and_Parser.ipynb
    exec(open('Data_Simulator_and_Parser.py').read())
else:
    # Renaming Some internal Parameter(s)
    groud_truth = "rSDE"
    test_size_ratio = train_test_ratio
    Ratio_fBM_to_typical_vol = 1
    output_dim = problem_dim
    T_begin = 0
    T_end = 1
    # Run da code
    %run Fractional_SDE/fractional_SDE_Simulator.ipynb
    %run Fractional_SDE/Data_Simulator_and_Parser___fractional_SDE.ipynb
#     exec(open('Fractional_SDE/fractional_SDE_Simulator.py').read())
#     exec(open('Fractional_SDE/Data_Simulator_and_Parser___fractional_SDE.py').read())

# Verbosity is nice
print("Generated Data:")
print("Number of Training Datums:"+str(X_train.shape[0]))
print("Number of Testing Datums:"+str(X_test.shape[0]))

  0%|          | 0/10 [00:00<?, ?it/s]

Generating/Prasing Data and Training MC-Oracle
Building Training + Testing Set - rough-SDE Ground-Truth


100%|██████████| 10/10 [00:40<00:00,  4.07s/it]

Generated Data:
Number of Training Datums:100
Number of Testing Datums:100


### Rescale

In [13]:
# Rescale
scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

## Run: Main (DNM) and Oracle Models

In [14]:
print("------------------------------")
print("Running script for main model!")
print("------------------------------")
%run Universal_Measure_Valued_Networks_Backend.ipynb
# exec(open('Universal_Measure_Valued_Networks_Backend.py').read())
print("------------------------------------")
print("Done: Running script for main model!")
print("------------------------------------")

------------------------------
Running script for main model!
------------------------------
Deep Feature Builder - Ready
Deep Classifier - Ready
Training Classifer Portion of Type-A Model
Fitting 2 folds for each of 1 candidates, totalling 2 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   2 | elapsed:   17.7s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   2 out of   2 | elapsed:   17.7s finished


Epoch 1/800
7/7 [==============================] - 0s 2ms/step - loss: 3.9194 - accuracy: 0.0100
Epoch 2/800
7/7 [==============================] - 0s 2ms/step - loss: 3.9071 - accuracy: 0.0300
Epoch 3/800
7/7 [==============================] - 0s 2ms/step - loss: 3.8993 - accuracy: 0.0000e+00
Epoch 4/800
7/7 [==============================] - 0s 2ms/step - loss: 3.8839 - accuracy: 0.0600
Epoch 5/800
7/7 [==============================] - 0s 3ms/step - loss: 3.8571 - accuracy: 0.0500
Epoch 6/800
7/7 [==============================] - 0s 2ms/step - loss: 3.8085 - accuracy: 0.0500
Epoch 7/800
7/7 [==============================] - 0s 2ms/step - loss: 3.7525 - accuracy: 0.0400
Epoch 8/800
7/7 [==============================] - 0s 2ms/step - loss: 3.6778 - accuracy: 0.0200
Epoch 9/800
7/7 [==============================] - 0s 2ms/step - loss: 3.5903 - accuracy: 0.0400
Epoch 10/800
7/7 [==============================] - 0s 2ms/step - loss: 3.4609 - accuracy: 0.0600
Epoch 11/800
7/7 [=======

7/7 [==============================] - 0s 2ms/step - loss: 1.5598 - accuracy: 0.4300
Epoch 85/800
7/7 [==============================] - 0s 2ms/step - loss: 1.6113 - accuracy: 0.3600
Epoch 86/800
7/7 [==============================] - 0s 2ms/step - loss: 1.6396 - accuracy: 0.3900
Epoch 87/800
7/7 [==============================] - 0s 2ms/step - loss: 1.5736 - accuracy: 0.3800
Epoch 88/800
7/7 [==============================] - 0s 2ms/step - loss: 1.6080 - accuracy: 0.3100
Epoch 89/800
7/7 [==============================] - 0s 2ms/step - loss: 1.5442 - accuracy: 0.3800
Epoch 90/800
7/7 [==============================] - 0s 2ms/step - loss: 1.5886 - accuracy: 0.3200
Epoch 91/800
7/7 [==============================] - 0s 2ms/step - loss: 1.5569 - accuracy: 0.4100
Epoch 92/800
7/7 [==============================] - 0s 2ms/step - loss: 1.4864 - accuracy: 0.4100
Epoch 93/800
7/7 [==============================] - 0s 2ms/step - loss: 1.5795 - accuracy: 0.3700
Epoch 94/800
7/7 [===============

7/7 [==============================] - 0s 2ms/step - loss: 1.5796 - accuracy: 0.3700
Epoch 168/800
7/7 [==============================] - 0s 2ms/step - loss: 1.3847 - accuracy: 0.4200
Epoch 169/800
7/7 [==============================] - 0s 2ms/step - loss: 1.2982 - accuracy: 0.3700
Epoch 170/800
7/7 [==============================] - 0s 2ms/step - loss: 1.2480 - accuracy: 0.3800
Epoch 171/800
7/7 [==============================] - 0s 2ms/step - loss: 1.1862 - accuracy: 0.5700
Epoch 172/800
7/7 [==============================] - 0s 2ms/step - loss: 1.2889 - accuracy: 0.4700
Epoch 173/800
7/7 [==============================] - 0s 2ms/step - loss: 1.4415 - accuracy: 0.4100
Epoch 174/800
7/7 [==============================] - 0s 2ms/step - loss: 1.3390 - accuracy: 0.3800
Epoch 175/800
7/7 [==============================] - 0s 2ms/step - loss: 1.3798 - accuracy: 0.4900
Epoch 176/800
7/7 [==============================] - 0s 2ms/step - loss: 1.3616 - accuracy: 0.4500
Epoch 177/800
7/7 [=====

...

7/7 [==============================] - 0s 2ms/step - loss: 0.8966 - accuracy: 0.6300
Epoch 414/800
7/7 [==============================] - 0s 2ms/step - loss: 0.9162 - accuracy: 0.5600
Epoch 415/800
7/7 [==============================] - 0s 2ms/step - loss: 0.8708 - accuracy: 0.6000
Epoch 416/800
7/7 [==============================] - 0s 2ms/step - loss: 0.9160 - accuracy: 0.6100
Epoch 417/800
7/7 [==============================] - 0s 2ms/step - loss: 0.7866 - accuracy: 0.6800
Epoch 418/800
7/7 [==============================] - 0s 2ms/step - loss: 0.7810 - accuracy: 0.6800
Epoch 419/800
7/7 [==============================] - 0s 2ms/step - loss: 0.8239 - accuracy: 0.6700
Epoch 420/800
7/7 [==============================] - 0s 2ms/step - loss: 0.9086 - accuracy: 0.6300
Epoch 421/800
7/7 [==============================] - 0s 2ms/step - loss: 0.8134 - accuracy: 0.6900
Epoch 422/800
7/7 [==============================] - 0s 2ms/step - loss: 0.9397 - accuracy: 0.5500
Epoch 423/800
7/7 [=====

7/7 [==============================] - 0s 2ms/step - loss: 0.8142 - accuracy: 0.6600
Epoch 496/800
7/7 [==============================] - 0s 2ms/step - loss: 0.7665 - accuracy: 0.7100
Epoch 497/800
7/7 [==============================] - 0s 2ms/step - loss: 0.7079 - accuracy: 0.7700
Epoch 498/800
7/7 [==============================] - 0s 2ms/step - loss: 0.8039 - accuracy: 0.6700
Epoch 499/800
7/7 [==============================] - 0s 3ms/step - loss: 0.6962 - accuracy: 0.7300
Epoch 500/800
7/7 [==============================] - 0s 2ms/step - loss: 0.8282 - accuracy: 0.5800
Epoch 501/800
7/7 [==============================] - 0s 2ms/step - loss: 0.7646 - accuracy: 0.6900
Epoch 502/800
7/7 [==============================] - 0s 2ms/step - loss: 0.7567 - accuracy: 0.6800
Epoch 503/800
7/7 [==============================] - 0s 2ms/step - loss: 0.7458 - accuracy: 0.6800
Epoch 504/800
7/7 [==============================] - 0s 2ms/step - loss: 0.7567 - accuracy: 0.7200
Epoch 505/800
7/7 [=====

7/7 [==============================] - 0s 3ms/step - loss: 0.6793 - accuracy: 0.6600
Epoch 578/800
7/7 [==============================] - 0s 2ms/step - loss: 0.7067 - accuracy: 0.7400
Epoch 579/800
7/7 [==============================] - 0s 2ms/step - loss: 0.6605 - accuracy: 0.7000
Epoch 580/800
7/7 [==============================] - 0s 2ms/step - loss: 0.8072 - accuracy: 0.7000
Epoch 581/800
7/7 [==============================] - 0s 2ms/step - loss: 1.0514 - accuracy: 0.5300
Epoch 582/800
7/7 [==============================] - 0s 2ms/step - loss: 1.0589 - accuracy: 0.5800
Epoch 583/800
7/7 [==============================] - 0s 2ms/step - loss: 1.1231 - accuracy: 0.5100
Epoch 584/800
7/7 [==============================] - 0s 2ms/step - loss: 0.8587 - accuracy: 0.5800
Epoch 585/800
7/7 [==============================] - 0s 2ms/step - loss: 0.7160 - accuracy: 0.7200
Epoch 586/800
7/7 [==============================] - 0s 3ms/step - loss: 0.7777 - accuracy: 0.6500
Epoch 587/800
7/7 [=====

7/7 [==============================] - 0s 3ms/step - loss: 0.7105 - accuracy: 0.7600
Epoch 660/800
7/7 [==============================] - 0s 2ms/step - loss: 0.6030 - accuracy: 0.7900
Epoch 661/800
7/7 [==============================] - 0s 2ms/step - loss: 0.6810 - accuracy: 0.7000
Epoch 662/800
7/7 [==============================] - 0s 2ms/step - loss: 0.6520 - accuracy: 0.6900
Epoch 663/800
7/7 [==============================] - 0s 3ms/step - loss: 0.6147 - accuracy: 0.7600
Epoch 664/800
7/7 [==============================] - 0s 2ms/step - loss: 0.6470 - accuracy: 0.7100
Epoch 665/800
7/7 [==============================] - 0s 2ms/step - loss: 0.5590 - accuracy: 0.7900
Epoch 666/800
7/7 [==============================] - 0s 2ms/step - loss: 0.5505 - accuracy: 0.8200
Epoch 667/800
7/7 [==============================] - 0s 2ms/step - loss: 0.5761 - accuracy: 0.7600
Epoch 668/800
7/7 [==============================] - 0s 2ms/step - loss: 0.5461 - accuracy: 0.7700
Epoch 669/800
7/7 [=====

7/7 [==============================] - 0s 2ms/step - loss: 0.7997 - accuracy: 0.6200
Epoch 742/800
7/7 [==============================] - 0s 2ms/step - loss: 0.8584 - accuracy: 0.6500
Epoch 743/800
7/7 [==============================] - 0s 2ms/step - loss: 0.9989 - accuracy: 0.6200
Epoch 744/800
7/7 [==============================] - 0s 2ms/step - loss: 0.8138 - accuracy: 0.6100
Epoch 745/800
7/7 [==============================] - 0s 2ms/step - loss: 0.9270 - accuracy: 0.6100
Epoch 746/800
7/7 [==============================] - 0s 2ms/step - loss: 0.7405 - accuracy: 0.6500
Epoch 747/800
7/7 [==============================] - 0s 2ms/step - loss: 0.6090 - accuracy: 0.7200
Epoch 748/800
7/7 [==============================] - 0s 2ms/step - loss: 0.9789 - accuracy: 0.6000
Epoch 749/800
7/7 [==============================] - 0s 2ms/step - loss: 0.9089 - accuracy: 0.5900
Epoch 750/800
7/7 [==============================] - 0s 2ms/step - loss: 0.8588 - accuracy: 0.6500
Epoch 751/800
7/7 [=====

### Evaluate Main and Oracle Model's Performance

In [15]:
%run Universal_Measure_Valued_Networks_Backend_EVALUATOR.ipynb
# exec(open(Universal_Measure_Valued_Networks_Backend_EVALUATOR.py).read())

  0%|          | 0/100 [00:00<?, ?it/s]

#--------------------#
 Get Training Error(s)
#--------------------#


  2%|▏         | 2/100 [00:00<00:05, 17.94it/s]

#-------------------------#
 Get Training Error(s): END
#-------------------------#
#----------------#
 Get Test Error(s)
#----------------#


100%|██████████| 100/100 [00:02<00:00, 41.78it/s]


#------------------------#
 Get Testing Error(s): END
#------------------------#
                                         DNM  MC-Oracle
W1-95L                           1014.083556   0.000000
W1                               1248.975605   0.000000
W1-95R                           1424.657623   0.000000
M-95L                               1.789474   0.000000
M                                   1.836156   0.000000
M-95R                               1.880722   0.000000
N_Par                          131250.000000   0.000000
Train_Time                         41.474068   3.665909
Test_Time/MC-Oracle_Test_Time       0.012630   1.000000


## 1) *Pointmass Benchmark(s)*
These benchmarks consist of subsets of $C(\mathbb{R}^d,\mathbb{R})$ which we lift to models in $C(\mathbb{R}^d,\cap_{1\leq q<\infty}\mathscr{P}_{q}(\mathbb{R}))$ via:
$$
\mathbb{R}^d \ni x \to f(x) \to \delta_{f(x)}\in \cap_{1\leq q<\infty}\mathcal{P}_{q}(\mathbb{R}).
$$

In [16]:
exec(open('CV_Grid.py').read())
# Notebook Mode:
# %run Evaluation.ipynb
# %run Benchmarks_Model_Builder_Pointmass_Based.ipynb
# Terminal Mode (Default):
exec(open('Evaluation.py').read())
exec(open('Benchmarks_Model_Builder_Pointmass_Based.py').read())

Deep Feature Builder - Ready
--------------
Training: ENET
--------------


100%|██████████| 100/100 [00:00<00:00, 3141.07it/s]

---------------------
Training: ENET - Done
---------------------
#------------#
 Get Error(s) 
#------------#
#-----------------#
 Get Error(s): END 
#-----------------#
#------------#
 Get Error(s) 
#------------#
#-----------------#
 Get Error(s): END 
#-----------------#
Updated DataFrame
                                         DNM  MC-Oracle         ENET
W1-95L                           1014.083556   0.000000            -
W1                               1248.975605   0.000000            -
W1-95R                           1424.657623   0.000000            -
M-95L                               1.789474   0.000000     0.660137
M                                   1.836156   0.000000     0.774678
M-95R                               1.880722   0.000000     0.892973
N_Par                          131250.000000   0.000000            4
Train_Time                         41.474068   3.665909  1.62086e+09
Test_Time/MC-Oracle_Test_Time       0.012630   1.000000  4.33054e-05
----------------


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   1 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Batch computation too fast (0.0387s.) Setting batch_size=2.
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    0.1s remaining:    0.1s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.4s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.4s finished
100%|██████████| 100/100 [00:00<00:00, 3089.34it/s]
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.


#------------#
 Get Error(s) 
#------------#
#-----------------#
 Get Error(s): END 
#-----------------#
#------------#
 Get Error(s) 
#------------#
#-----------------#
 Get Error(s): END 
#-----------------#
Updated DataFrame
                                         DNM  MC-Oracle         ENET  \
W1-95L                           1014.083556   0.000000            -   
W1                               1248.975605   0.000000            -   
W1-95R                           1424.657623   0.000000            -   
M-95L                               1.789474   0.000000     0.660137   
M                                   1.836156   0.000000     0.774678   
M-95R                               1.880722   0.000000     0.892973   
N_Par                          131250.000000   0.000000            4   
Train_Time                         41.474068   3.665909  1.62086e+09   
Test_Time/MC-Oracle_Test_Time       0.012630   1.000000  4.33054e-05   

                                    KRidge  
W1-95L

...

Epoch 790/800
7/7 [==============================] - 0s 2ms/step - loss: 0.4120 - mse: 0.2749 - mae: 0.4120 - mape: 82.7962
Epoch 791/800
7/7 [==============================] - 0s 2ms/step - loss: 0.4078 - mse: 0.3077 - mae: 0.4078 - mape: 67.8662
Epoch 792/800
7/7 [==============================] - 0s 2ms/step - loss: 0.4082 - mse: 0.2861 - mae: 0.4082 - mape: 76.1770
Epoch 793/800
7/7 [==============================] - 0s 2ms/step - loss: 0.4089 - mse: 0.2619 - mae: 0.4089 - mape: 87.1009
Epoch 794/800
7/7 [==============================] - 0s 2ms/step - loss: 0.4126 - mse: 0.3029 - mae: 0.4126 - mape: 70.7167
Epoch 795/800
7/7 [==============================] - 0s 2ms/step - loss: 0.4059 - mse: 0.2828 - mae: 0.4059 - mape: 76.5767
Epoch 796/800
7/7 [==============================] - 0s 3ms/step - loss: 0.4111 - mse: 0.2628 - mae: 0.4111 - mape: 87.3606
Epoch 797/800
7/7 [==============================] - 0s 2ms/step - loss: 0.4110 - mse: 0.2583 - mae: 0.4110 - mape: 90.9612
Epoch 79

  0%|          | 0/100 [00:00<?, ?it/s]

#------------#
 Get Error(s) 
#------------#


100%|██████████| 100/100 [00:00<00:00, 2263.88it/s]

#-----------------#
 Get Error(s): END 
#-----------------#
#------------#
 Get Error(s) 
#------------#
#-----------------#
 Get Error(s): END 
#-----------------#
Updated DataFrame
                                         DNM  MC-Oracle         ENET  \
W1-95L                           1014.083556   0.000000            -   
W1                               1248.975605   0.000000            -   
W1-95R                           1424.657623   0.000000            -   
M-95L                               1.789474   0.000000     0.660137   
M                                   1.836156   0.000000     0.774678   
M-95R                               1.880722   0.000000     0.892973   
N_Par                          131250.000000   0.000000            4   
Train_Time                         41.474068   3.665909  1.62086e+09   
Test_Time/MC-Oracle_Test_Time       0.012630   1.000000  4.33054e-05   

                                    KRidge         GBRF        DNN  
W1-95L                     

# Summary of Point-Mass Regression Models

#### Training Model Facts

In [17]:
print(Summary_pred_Qual_models)
Summary_pred_Qual_models

                                         DNM  MC-Oracle         ENET  \
W1-95L                           1014.083556   0.000000            -   
W1                               1248.975605   0.000000            -   
W1-95R                           1424.657623   0.000000            -   
M-95L                               1.789474   0.000000     0.660137   
M                                   1.836156   0.000000     0.774678   
M-95R                               1.880722   0.000000     0.892973   
N_Par                          131250.000000   0.000000            4   
Train_Time                         41.474068   3.665909  1.62086e+09   
Test_Time/MC-Oracle_Test_Time       0.012630   1.000000  4.33054e-05   

                                    KRidge         GBRF        DNN  
W1-95L                                   -            -          -  
W1                                       -            -          -  
W1-95R                                   -            -          -  
M-9

,DNM,MC-Oracle,ENET,KRidge,GBRF,DNN
W1-95L,1014.083556,0.000000,-,-,-,-
W1,1248.975605,0.000000,-,-,-,-
W1-95R,1424.657623,0.000000,-,-,-,-
M-95L,1.789474,0.000000,0.660137,8.48656e+42,0.702467,0.623415
M,1.836156,0.000000,0.774678,8.48656e+42,0.747113,0.667836
M-95R,1.880722,0.000000,0.892973,8.48656e+42,0.816436,0.722768
N_Par,131250.000000,0.000000,4,0,616,121401
Train_Time,41.474068,3.665909,1.62086e+09,0.566994,0.134516,39.4043
Test_Time/MC-Oracle_Test_Time,0.012630,1.000000,4.33054e-05,0.00130782,4.80138e-05,0.0124975


#### Testing Model Facts

In [18]:
print(Summary_pred_Qual_models_test)
Summary_pred_Qual_models_test

                                         DNM  MC-Oracle          ENET  \
W1-95L                           1014.083556   0.000000  6.290373e+02   
W1                               1248.975605   0.000000  7.934841e+02   
W1-95R                           1424.657623   0.000000  9.379718e+02   
M-95L                               1.789474   0.000000  6.601372e-01   
M                                   1.836156   0.000000  7.746780e-01   
M-95R                               1.880722   0.000000  8.929732e-01   
N_Par                          131250.000000   0.000000  4.000000e+00   
Train_Time                         41.474068   3.665909  1.620860e+09   
Test_Time/MC-Oracle_Test_Time       0.012630   1.000000  4.330542e-05   

                                     KRidge        GBRF            DNN  
W1-95L                         7.202173e+85  619.750000     597.783576  
W1                             7.202173e+85  793.090502     792.955186  
W1-95R                         7.202173e+85  984.6

,DNM,MC-Oracle,ENET,KRidge,GBRF,DNN
W1-95L,1014.083556,0.000000,6.290373e+02,7.202173e+85,619.750000,597.783576
W1,1248.975605,0.000000,7.934841e+02,7.202173e+85,793.090502,792.955186
W1-95R,1424.657623,0.000000,9.379718e+02,7.202173e+85,984.664445,985.873339
M-95L,1.789474,0.000000,6.601372e-01,8.486561e+42,0.702467,0.623415
M,1.836156,0.000000,7.746780e-01,8.486561e+42,0.747113,0.667836
M-95R,1.880722,0.000000,8.929732e-01,8.486561e+42,0.816436,0.722768
N_Par,131250.000000,0.000000,4.000000e+00,0.000000e+00,616.000000,121401.000000
Train_Time,41.474068,3.665909,1.620860e+09,5.669942e-01,0.134516,39.404312
Test_Time/MC-Oracle_Test_Time,0.012630,1.000000,4.330542e-05,1.307824e-03,0.000048,0.012497


## 2) *Gaussian Benchmarks*

- Bencharm 1: [Gaussian Process Regressor](https://scikit-learn.org/stable/modules/gaussian_process.html)
- Benchmark 2: Deep Gaussian Networks:
These models train models which assume Gaussianity.  We may view these as models in $\mathcal{P}_2(\mathbb{R})$ via:
$$
\mathbb{R}^d \ni x \to (\hat{\mu}(x),\hat{\Sigma}(x)\hat{\Sigma}^{\top})\triangleq f(x) \in \mathbb{R}\times [0,\infty) \to 
(2\pi)^{-\frac{d}{2}}\det(\hat{\Sigma}(x))^{-\frac{1}{2}} \, e^{ -\frac{1}{2}(\cdot - \hat{\mu}(x))^{{{\!\mathsf{T}}}} \hat{\Sigma}(x)^{-1}(\cdot - \hat{\mu}(x)) } \mu \in \mathcal{G}_d\subset \mathcal{P}_2(\mathbb{R});
$$
where $\mathcal{G}_1$ is the set of Gaussian measures on $\mathbb{R}$ equipped with the relative Wasserstein-1 topology.

Examples of this type of architecture are especially prevalent in uncertainty quantification; see ([Deep Ensembles](https://arxiv.org/abs/1612.01474)] or [NOMU: Neural Optimization-based Model Uncertainty](https://arxiv.org/abs/2102.13640).  Moreover, their universality in $C(\mathbb{R}^d,\mathcal{G}_2)$ is known, and has been shown in [Corollary 4.7](https://arxiv.org/abs/2101.05390).

In [19]:
# %run Benchmarks_Model_Builder_Mean_Var.ipynb
exec(open('Benchmarks_Model_Builder_Mean_Var.py').read())

Deep Feature Builder - Ready
Fitting 2 folds for each of 2 candidates, totalling 4 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   1 tasks      | elapsed:    0.4s
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    0.5s remaining:    0.5s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.9s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.9s finished
100%|██████████| 100/100 [00:00<00:00, 1653.60it/s]

Infering Parameters for Deep Gaussian Network to train on!
Done Getting Parameters for Deep Gaussian Network!
Training Deep Gaussian Network!
Fitting 2 folds for each of 1 candidates, totalling 2 fits



[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   2 | elapsed:   15.2s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   2 out of   2 | elapsed:   15.2s finished


Epoch 1/800
7/7 [==============================] - 0s 2ms/step - loss: nan - mse: nan - mae: nan - mape: nan           
Epoch 2/800
7/7 [==============================] - 0s 2ms/step - loss: nan - mse: nan - mae: nan - mape: nan
Epoch 3/800
7/7 [==============================] - 0s 2ms/step - loss: nan - mse: nan - mae: nan - mape: nan
Epoch 4/800
7/7 [==============================] - 0s 2ms/step - loss: nan - mse: nan - mae: nan - mape: nan
Epoch 5/800
7/7 [==============================] - 0s 2ms/step - loss: nan - mse: nan - mae: nan - mape: nan
Epoch 6/800
7/7 [==============================] - 0s 2ms/step - loss: nan - mse: nan - mae: nan - mape: nan
Epoch 7/800
7/7 [==============================] - 0s 2ms/step - loss: nan - mse: nan - mae: nan - mape: nan
Epoch 8/800
7/7 [==============================] - 0s 2ms/step - loss: nan - mse: nan - mae: nan - mape: nan
Epoch 9/800
7/7 [==============================] - 0s 2ms/step - loss: nan - mse: nan - mae: nan - mape: nan
Epoch 10

...

7/7 [==============================] - 0s 3ms/step - loss: nan - mse: nan - mae: nan - mape: nan
Epoch 661/800
7/7 [==============================] - 0s 4ms/step - loss: nan - mse: nan - mae: nan - mape: nan
Epoch 662/800
7/7 [==============================] - 0s 4ms/step - loss: nan - mse: nan - mae: nan - mape: nan
Epoch 663/800
7/7 [==============================] - 0s 3ms/step - loss: nan - mse: nan - mae: nan - mape: nan
Epoch 664/800
7/7 [==============================] - 0s 3ms/step - loss: nan - mse: nan - mae: nan - mape: nan
Epoch 665/800
7/7 [==============================] - 0s 2ms/step - loss: nan - mse: nan - mae: nan - mape: nan
Epoch 666/800
7/7 [==============================] - 0s 3ms/step - loss: nan - mse: nan - mae: nan - mape: nan
Epoch 667/800
7/7 [==============================] - 0s 3ms/step - loss: nan - mse: nan - mae: nan - mape: nan
Epoch 668/800
7/7 [==============================] - 0s 3ms/step - loss: nan - mse: nan - mae: nan - mape: nan
Epoch 669/800
7

7/7 [==============================] - 0s 3ms/step - loss: nan - mse: nan - mae: nan - mape: nan
Epoch 734/800
7/7 [==============================] - 0s 3ms/step - loss: nan - mse: nan - mae: nan - mape: nan
Epoch 735/800
7/7 [==============================] - 0s 3ms/step - loss: nan - mse: nan - mae: nan - mape: nan
Epoch 736/800
7/7 [==============================] - 0s 3ms/step - loss: nan - mse: nan - mae: nan - mape: nan
Epoch 737/800
7/7 [==============================] - 0s 3ms/step - loss: nan - mse: nan - mae: nan - mape: nan
Epoch 738/800
7/7 [==============================] - 0s 3ms/step - loss: nan - mse: nan - mae: nan - mape: nan
Epoch 739/800
7/7 [==============================] - 0s 4ms/step - loss: nan - mse: nan - mae: nan - mape: nan
Epoch 740/800
7/7 [==============================] - 0s 3ms/step - loss: nan - mse: nan - mae: nan - mape: nan
Epoch 741/800
7/7 [==============================] - 0s 3ms/step - loss: nan - mse: nan - mae: nan - mape: nan
Epoch 742/800
7

  0%|          | 0/100 [00:00<?, ?it/s]

Training Deep Gaussian Network!: END
#---------------------------------------#
 Get Training Errors for: Gaussian Models
#---------------------------------------#


ValueError: scale < 0

In [ ]:
print("Prediction Quality (Updated): Test")
print(Summary_pred_Qual_models_test)
Summary_pred_Qual_models_test

In [ ]:
print("Prediction Quality (Updated): Train")
print(Summary_pred_Qual_models)
Summary_pred_Qual_models

# 3) The natural Universal Benchmark: [Bishop's Mixture Density Network](https://publications.aston.ac.uk/id/eprint/373/1/NCRG_94_004.pdf)

This implementation is as follows:
- For every $x$ in the trainingdata-set we fit a GMM $\hat{\nu}_x$, using the [Expectation-Maximization (EM) algorithm](https://en.wikipedia.org/wiki/Expectation%E2%80%93maximization_algorithm), with the same number of centers as the deep neural model in $\mathcal{NN}_{1_{\mathbb{R}^d},\mathcal{D}}^{\sigma:\star}$ which we are evaluating.  
- A Mixture density network is then trained to predict the infered parameters; given any $x \in \mathbb{R}^d$.

In [ ]:
if output_dim == 1:
    # %run Mixture_Density_Network.ipynb
    exec(open('Mixture_Density_Network.py').read())

## Get Final Outputs
Now we piece together all the numerical experiments and report a nice summary.

---
# Final Results
---

## Prasing Quality Metric Results

#### Finalizing Saving
**Note:** *We do it in two steps since the grid sometimes does not want to write nicely...*

In [ ]:
## Write Performance Metrics
### Incase caption breaks
Summary_pred_Qual_models.to_latex((results_tables_path+"/Final_Results/"+"Performance_metrics_Problem_Type_"+str(f_unknown_mode)+"Problemdimension"+str(problem_dim)+"__SUMMARY_METRICS.tex"),
                                 float_format="{:0.3g}".format)
text_file = open((results_tables_path+"/Final_Results/"+"ZZZ_CAPTION_Performance_metrics_Problem_Type_"+str(f_unknown_mode)+"Problemdimension"+str(problem_dim)+"__SUMMARY_METRICS___CAPTION.tex"), "w")
text_file.write("Quality Metrics; d:"+str(problem_dim)+", D:"+str(output_dim)+", Depth:"+str(Depth_Bayesian_DNN)+", Width:"+str(width)+", Dropout rate:"+str(Dropout_rate)+".")
text_file.close()


### Incase caption does not break
Summary_pred_Qual_models.to_latex((results_tables_path+"/Final_Results/"+"Performance_metrics_Problem_Type_"+str(f_unknown_mode)+"Problemdimension"+str(problem_dim)+"__SUMMARY_METRICS.tex"),
                                 caption=("Quality Metrics; d:"+str(problem_dim)+", D:"+str(output_dim)+", Depth:"+str(Depth_Bayesian_DNN)+", Width:"+str(width)+", Dropout rate:"+str(Dropout_rate)+"."),
                                 float_format="{:0.3g}".format)

# For Terminal Runner(s):

In [ ]:
# For Terminal Running
print("===================")
print("Predictive Quality:")
print("===================")
print(Summary_pred_Qual_models)
print("===================")
print(" ")
print(" ")
print(" ")
print("Kernel_Used_in_GPR: "+str(GPR_trash.kernel))
print("🙃🙃 Have a wonderful day! 🙃🙃")
Summary_pred_Qual_models

---
# Fin
---

---